In [2]:
# import the tmax data: 

import numpy as np
import pandas as pd

# read in the datafile: 

data=pd.read_csv("/home/svetlanayegorova/Documents/UM_Autumn_2020/Machine_learning/Project/Temp_t.csv")
print(data.keys())
print(data.shape)
type(data)

# think of  how to pass the data to the model. 

Index(['264.5', '266.7', '272.8', '274.5', '272', '268.1', '259.8', '259.8.1',
       '270.6', '270.8',
       ...
       '269', '272.5.3', '271.7.4', '273.3', '273.5.1', '274.1', '271.5',
       '268.3.1', '265.9', '263.4.2'],
      dtype='object', length=365)
(29626, 365)


pandas.core.frame.DataFrame

In [3]:
# import pytorch and the needed libraries: 

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F

Specify the network. Compress data to two dimentions in the latent space

In [98]:
features = 2 # number of features in the latent space. 

# define a simple linear VAE
class LinearVAE(nn.Module):
    def __init__(self):
        super(LinearVAE, self).__init__()
 
        # encoder
        self.enc1 = nn.Linear(in_features=365, out_features=180) # daily temp as the first input
        self.enc2 = nn.Linear(in_features=180, out_features=features*2) #  multiply number of features by  because? 
 
        # decoder 
        self.dec1 = nn.Linear(in_features=features, out_features=180)
        self.dec2 = nn.Linear(in_features=180, out_features=365)
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling as if coming from the input space
        return sample
 
    def forward(self, x):
        # encoding
        x = F.relu(self.enc1(x))
        x = self.enc2(x).view(-1, 2, features)
        #x = self.enc2(x)
        
        # get `mu` and `log_var`
        mu = x[:, 0, :] # the first feature values as mean
        log_var = x[:, 1, :] # the other feature values as variance
        # get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
 
        # decoding
        x = F.relu(self.dec1(z))
        reconstruction = torch.sigmoid(self.dec2(x))
        return reconstruction, mu, log_var

In [88]:
# import torch tools: 
import torch
import torch.nn as nn
import torch.nn.functional as F

 -divide data into training and testing poritons
 -transform data to a tensor 
 -define training and testing data loaders
 -initialize the model
 -define optimizer
 -define criterion (MSE)
 -define final loss function that combines KL divergence and the criterion
 
 -define training funciton for the model
 -define test/validate the model


In [89]:
# -divide data into training and testing poritons

from sklearn.model_selection import train_test_split
X, X_test=train_test_split(data)
print("train shape", X.shape)
print("test shape", X_test.shape)

train shape (22219, 365)
test shape (7407, 365)


In [104]:
# transform data to tensors: 
X=torch.from_numpy(np.array(X))
X=X.to(torch.float32)

X_test=torch.from_numpy(np.array(X_test))
X_test=X_test.to(torch.float32)


In [92]:
# define training and testing data loaders: 
from torch.utils.data import TensorDataset

training_data = TensorDataset(X, X)
test_data = TensorDataset(X_test, X_test)



In [105]:


batch_size = 25
train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                           batch_size=batch_size, 
                                           shuffle=True)

batch_size = 25
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                           batch_size=batch_size, 
                                           shuffle=False)

In [39]:
print(train_loader)

In [106]:
# -initialize the model
#  -define optimizer
#  -define criterion
model = LinearVAE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.MSELoss(reduction='sum') # USE Mean Squared Error, or really sum of squared error as the loss function



In [107]:
#  -define final loss function that combines KL divergence and the criterion


def final_loss(mse_loss, mu, logvar):
    """
    This function will add the reconstruction loss (MSELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    MSE = mse_loss 
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE + KLD

In [109]:
#  -define training funciton for the model
#  -define test/validate the model

epochs=25
running_loss=0.0
# Loop over the data
for epoch in range(epochs):
    # Loop over each subset of data
    for d, t in train_loader: # d and t are data and target..
        # Zero out the optimizer's gradient buffer
        optimizer.zero_grad()
        
        # Make a prediction based on the model
        reconstruction, mu, logvar = model(d)
        
        # Compute the mse loss
        mse_loss = criterion(reconstruction, t)
        print(mse_loss)
        
        # Compute the total loss, including KL div:
        loss=final_loss(mse_loss, mu, logvar)
        #print(loss)
        
        # keep track of the running loss: 
        running_loss+=loss.item()
        #print(running_loss)
      

        # Use backpropagation to compute the derivative of the loss with respect to the parameters
        loss.backward()
        
        # Use the derivative information to update the parameters
        optimizer.step()
        
        
### at the end of training, expet to have the parameters

tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseL

tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseL

tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseL

tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseL

tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseL

...

tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseL

tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseL

tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseL

tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseL

tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseLossBackward>)
tensor(nan, grad_fn=<MseL

In [103]:
# do a manual walk-through
print(model(d))

(tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<SigmoidBackward>), tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan]], grad_fn=<SliceBackward>), tensor([[nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, nan],
        [nan, 